# Merged BTS and Weather Data

In [ ]:
# Faster solution for multiple years worth of data for delays
import requests
import pandas as pd
import zipfile
import io
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

def download_bts_data(year, month):
    """Download BTS data for a specific month"""
    url = f'https://www.transtats.bts.gov/PREZIP/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
    
    try:
        response = requests.get(
            url, 
            headers={'User-Agent': 'Mozilla/5.0'},
            timeout=300,
            stream=True  # Stream large files
        )
        response.raise_for_status()
        
        # Extract CSV from zip
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            csv_file = next((name for name in z.namelist() if name.endswith('.csv')), None)
            if not csv_file:
                return (year, month, None, "No CSV in zip")
            
            with z.open(csv_file) as f:
                df = pd.read_csv(f, encoding='utf-8', low_memory=False)
            
            return (year, month, df, None)
            
    except requests.exceptions.HTTPError as e:
        return (year, month, None, f"HTTP {e.response.status_code}")
    except Exception as e:
        return (year, month, None, str(e)[:50])

def download_years_parallel(start_year, end_year, max_workers=20):
    """
    Download multiple years of data with maximum parallelization
    
    Args:
        start_year: Starting year (e.g., 2005)
        end_year: Ending year (e.g., 2024)
        max_workers: Number of parallel downloads (default: 20)
    
    Returns:
        Combined DataFrame
    """
    # Generate all year-month combinations
    months_to_download = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            # Skip future months
            current_date = datetime.now()
            if year > current_date.year or (year == current_date.year and month > current_date.month):
                continue
            months_to_download.append((year, month))
    
    total_months = len(months_to_download)
    print(f"Downloading {total_months} months ({start_year}-{end_year})")
    print(f"Using {max_workers} parallel workers")
    print(f"Estimated time: {(total_months * 2.5) / max_workers:.1f} minutes\n")
    
    results = {}
    completed = 0
    start_time = datetime.now()
    
    # Download all months in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(download_bts_data, year, month): (year, month) 
            for year, month in months_to_download
        }
        
        for future in as_completed(futures):
            year, month, df, error = future.result()
            completed += 1
            
            if df is not None:
                results[(year, month)] = df
                print(f"✓ {year}-{month:02d} ({len(df):,} rows) [{completed}/{total_months}]")
            else:
                results[(year, month)] = None
                print(f"✗ {year}-{month:02d} - {error} [{completed}/{total_months}]")
    
    elapsed = (datetime.now() - start_time).total_seconds() / 60
    
    # Sort and combine successful downloads
    successful = [(k, v) for k, v in sorted(results.items()) if v is not None]
    failed = [(k, v) for k, v in sorted(results.items()) if v is None]
    
    print(f"\n{'='*70}")
    print(f"Download Complete in {elapsed:.1f} minutes")
    print(f"  Successful: {len(successful)}/{total_months} months")
    if failed:
        print(f"  Failed: {len(failed)} months - {[f'{y}-{m:02d}' for (y,m), _ in failed[:10]]}")
    print(f"{'='*70}\n")
    
    if successful:
        print("Combining all data...")
        combine_start = datetime.now()
        
        # Combine all dataframes at once
        combined_df = pd.concat([df for _, df in successful], ignore_index=True, copy=False)
        
        combine_time = (datetime.now() - combine_start).total_seconds()
        print(f"✓ Combined {len(combined_df):,} rows in {combine_time:.1f}s")
        return combined_df
    else:
        print("✗ No data downloaded")
        return None

def save_to_csv_fast(df, filename):
    """Save DataFrame to CSV with progress"""
    print(f"\nSaving to {filename}...")
    start = datetime.now()
    
    # Save with efficient settings
    df.to_csv(filename, index=False)
    
    elapsed = (datetime.now() - start).total_seconds()
    file_size = os.path.getsize(filename) / (1024 * 1024)
    
    print(f"✓ Saved in {elapsed:.1f}s ({file_size:.1f} MB)")

# Main execution
if __name__ == "__main__":
    print("="*70)
    print("BTS On-Time Performance Data - ULTRA FAST DOWNLOADER")
    print("="*70 + "\n")
    
    # Configure your date range here
    START_YEAR = 2005
    END_YEAR = 2024
    MAX_WORKERS = 12  # Balanced: fast but respectful to server
    
    print(f"Configuration:")
    print(f"  Years: {START_YEAR} to {END_YEAR}")
    print(f"  Parallel workers: {MAX_WORKERS}")
    print(f"  Note: 240 months = 20 years (2005-2024)\n")
    
    # Download all data
    combined_data = download_years_parallel(START_YEAR, END_YEAR, max_workers=MAX_WORKERS)
    
    if combined_data is not None:
        output_file = f"bts_ontime_{START_YEAR}_{END_YEAR}_combined.csv"
        save_to_csv_fast(combined_data, output_file)
        
        print(f"\n{'='*70}")
        print(f"✓ SUCCESS!")
        print(f"{'='*70}")
        print(f"  File: {output_file}")
        print(f"  Total rows: {len(combined_data):,}")
        print(f"  Columns: {len(combined_data.columns)}")
        print(f"  Date range: {combined_data['FL_DATE'].min()} to {combined_data['FL_DATE'].max()}")
        print(f"\nFirst 10 columns:")
        for i, col in enumerate(combined_data.columns[:10], 1):
            print(f"  {i}. {col}")
        print(f"{'='*70}")
    else:
        print("\n" + "="*70)
        print("✗ FAILED - No data downloaded")
        print("="*70)

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Reading 2024 data
df = pd.read_csv("bts_ontime_2024_combined.csv")

percent_M = df.isna().mean() * 100

print(percent_M)


FL_DATE                   0.000000
OP_UNIQUE_CARRIER         0.000000
ORIGIN_AIRPORT_ID         0.000000
ORIGIN_AIRPORT_SEQ_ID     0.000000
ORIGIN_CITY_MARKET_ID     0.000000
ORIGIN                    0.000000
ORIGIN_CITY_NAME          0.000000
ORIGIN_STATE_ABR          0.000000
ORIGIN_STATE_FIPS         0.000000
ORIGIN_STATE_NM           0.000000
ORIGIN_WAC                0.000000
DEST_AIRPORT_ID           0.000000
DEST_AIRPORT_SEQ_ID       0.000000
DEST_CITY_MARKET_ID       0.000000
DEST                      0.000000
DEST_CITY_NAME            0.000000
DEST_STATE_ABR            0.000000
DEST_STATE_FIPS           0.000000
DEST_STATE_NM             0.000000
DEST_WAC                  0.000000
CRS_DEP_TIME              0.000000
DEP_TIME                  1.661803
DEP_DELAY                 1.669160
DEP_DELAY_NEW             1.669160
DEP_DEL15                 1.669160
DEP_DELAY_GROUP           1.669160
DEP_TIME_BLK              0.000000
TAXI_OUT                  1.706187
WHEELS_OFF          

In [39]:
# Combined weather data
import os
import pandas as pd
import requests
import glob

# -----------------------------
# Airport → GHCN‑Daily Station ID mapping
# -----------------------------
stations = {
    "ABQ": "USW00023050",
    "AMA": "USW00023020",
    "ATL": "USW00013874",
    "AUS": "USW00013958",
    "BWI": "USW00093721",
    "BUR": "USW00093195",
    "CHS": "USW00093824",
    "CLE": "USW00014830",
    "CMH": "USW00094846",
    "DAL": "USW00013960",
    "DEN": "USW00023040",
    "HOU": "USW00012918",
    "LAS": "USW00024049",
    "LAX": "USW00023174",
    "MDW": "USW00094846",
    "MCO": "USW00012839",
    "OAK": "USW00023272",
    "PHX": "USW00023183",
    "SAN": "USW00023188",
    "SFO": "USW00023234",
    "SJC": "USW00023282",
    "SMF": "USW00023266",
    "STL": "USW00093928",
    "TPA": "USW00012839",
    # Add more airports and station IDs as needed
}

BASE_URL = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/all/{}.dly"
OUTPUT_DIR = "sw_airport_daily_weather_bulk"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Variables to extract
VARS = {"TMAX", "TMIN", "PRCP", "AWND"}

def parse_ghcn_dly(file_path):
    rows = []
    with open(file_path, "r") as f:
        for line in f:
            year = int(line[11:15])
            month = int(line[15:17])
            variable = line[17:21]
            if variable not in VARS:
                continue
            for day in range(31):
                raw = line[21 + day*8:26 + day*8]
                try:
                    value = int(raw)
                except:
                    continue
                if value == -9999:
                    continue
                date = f"{year}-{month:02d}-{day+1:02d}"
                if variable in ["TMAX", "TMIN"]:
                    val = value / 10.0
                elif variable == "PRCP":
                    val = value / 10.0
                elif variable == "AWND":
                    val = value / 10.0
                else:
                    val = value
                rows.append({"DATE": date, "VARIABLE": variable, "VALUE": val})
    df = pd.DataFrame(rows)
    df = df.pivot(index="DATE", columns="VARIABLE", values="VALUE").reset_index()
    return df

# -----------------------------
# Download, parse, save per airport
# -----------------------------
for airport, stn in stations.items():
    url = BASE_URL.format(stn)
    local_file = os.path.join(OUTPUT_DIR, f"{stn}.dly")
    
    if not os.path.exists(local_file):
        print(f"Downloading {airport} ({stn}) ...")
        r = requests.get(url, timeout=60)
        if r.status_code != 200:
            print(f"❌ Failed to download station {stn} for airport {airport}")
            continue
        with open(local_file, "wb") as f:
            f.write(r.content)
    else:
        print(f"✅ Station file already exists for {airport} ({stn})")
    
    print(f"Parsing data for {airport} ({stn}) ...")
    try:
        df = parse_ghcn_dly(local_file)
        df["AIRPORT"] = airport  # add airport column
        out_csv = os.path.join(OUTPUT_DIR, f"{airport}_daily_weather.csv")
        df.to_csv(out_csv, index=False)
        print(f"Saved CSV for {airport}: {out_csv}")
    except Exception as e:
        print(f"❌ Error parsing {airport} ({stn}): {e}")

# -----------------------------
# Combine all airport CSVs into one bulk CSV
# -----------------------------
print("Combining all airport CSVs into one bulk CSV...")
all_files = glob.glob(os.path.join(OUTPUT_DIR, "*_daily_weather.csv"))
df_all = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
bulk_csv_path = os.path.join(OUTPUT_DIR, "SW_airports_all_daily_weather.csv")
df_all.to_csv(bulk_csv_path, index=False)
print(f"🎯 Bulk CSV saved: {bulk_csv_path}")
print(df_all.head())


Parsing data for ABQ (USW00023050) ...
Parsing data for ABQ (USW00023050) ...
Saved CSV for ABQ: sw_airport_daily_weather_bulk\ABQ_daily_weather.csv
Saved CSV for ABQ: sw_airport_daily_weather_bulk\ABQ_daily_weather.csv
Parsing data for AMA (USW00023020) ...
Parsing data for AMA (USW00023020) ...
Saved CSV for AMA: sw_airport_daily_weather_bulk\AMA_daily_weather.csv
Saved CSV for AMA: sw_airport_daily_weather_bulk\AMA_daily_weather.csv
Parsing data for ATL (USW00013874) ...
Parsing data for ATL (USW00013874) ...
Saved CSV for ATL: sw_airport_daily_weather_bulk\ATL_daily_weather.csv
Saved CSV for ATL: sw_airport_daily_weather_bulk\ATL_daily_weather.csv
Parsing data for AUS (USW00013958) ...
Parsing data for AUS (USW00013958) ...
Saved CSV for AUS: sw_airport_daily_weather_bulk\AUS_daily_weather.csv
Saved CSV for AUS: sw_airport_daily_weather_bulk\AUS_daily_weather.csv
Parsing data for BWI (USW00093721) ...
Parsing data for BWI (USW00093721) ...
Saved CSV for BWI: sw_airport_daily_weathe

In [3]:
from meteostat import Point, Daily
import pandas as pd
from datetime import datetime
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# ---------------- CONFIG ----------------
SWA_AIRPORTS = {
    "DAL": (32.8471, -96.8517),
    "AUS": (30.1975, -97.6664),
    "HOU": (29.6454, -95.2789),
    "SAT": (29.5337, -98.4698),
    "ELP": (31.8070, -106.3779),
    "LBB": (33.6609, -101.8214),
    "MAF": (31.9369, -102.2016),
    "HRL": (26.2285, -97.6544),
    "LAX": (33.9425, -118.4081),
    "OAK": (37.7126, -122.2197),
    "SAN": (32.7338, -117.1933),
    "SJC": (37.3639, -121.9289),
    "SMF": (38.6950, -121.5908),
    "BUR": (34.2007, -118.3587),
    "ONT": (34.0559, -117.6009),
    "SNA": (33.6757, -117.8682),
    "MCO": (28.4312, -81.3081),
    "TPA": (27.9755, -82.5332),
    "FLL": (26.0726, -80.1527),
    "PBI": (26.6832, -80.0956),
    "RSW": (26.5362, -81.7552),
    "PNS": (30.4734, -87.1867),
    "JAX": (30.4941, -81.6879),
    "PHX": (33.4342, -112.0116),
    "TUS": (32.1161, -110.9410),
    "DEN": (39.8561, -104.6737),
    "LAS": (36.0801, -115.1522),
    "RNO": (39.4993, -119.7681),
    "MDW": (41.7868, -87.7522),
    "ATL": (33.6407, -84.4277),
    "BWI": (39.1754, -76.6684),
    "STL": (38.7487, -90.3700),
    "MCI": (39.2976, -94.7139),
    "BNA": (36.1245, -86.6782),
    "MEM": (35.0425, -89.9767),
    "ABQ": (35.0494, -106.6172),
    "OKC": (35.3931, -97.6008),
    "TUL": (36.1986, -95.8880),
    "MSY": (29.9934, -90.2580),
    "RDU": (35.8776, -78.7875),
    "CLT": (35.2140, -80.9431),
    "PHL": (39.8719, -75.2411),
    "PIT": (40.4915, -80.2328),
    "BOS": (42.3656, -71.0096),
    "BUF": (42.9405, -78.7322),
    "ALB": (42.7483, -73.8026),
    "ROC": (43.1181, -77.6721),
    "ISP": (40.7953, -73.1000),
    "CMH": (39.9973, -82.8876),
    "CLE": (41.4117, -81.8498),
    "IND": (39.7173, -86.2944),
    "MSP": (44.8848, -93.2223),
    "MKE": (42.9473, -87.8960),
    "DTW": (42.2162, -83.3554),
    "GRR": (42.8808, -85.5228),
    "SLC": (40.7884, -111.9777),
    "SEA": (47.4490, -122.3093),
    "GEG": (47.6269, -117.5331),
    "PDX": (45.5898, -122.5951),
    "ICT": (37.6528, -97.4331),
    "LIT": (34.7275, -92.2241),
    "SDF": (38.1740, -85.7368),
    "DSM": (41.5341, -93.6600),
    "OMA": (41.3025, -95.8941),
    "BHM": (33.5629, -86.7536),
    "RIC": (37.5052, -77.3191),
    "ORF": (36.8946, -76.2015),
    "BDL": (41.9389, -72.6839),
    "PVD": (41.7246, -71.4283),
    "MHT": (42.9327, -71.4350)
}

OUTPUT_DIR = Path("meteostat_daily")
OUTPUT_DIR.mkdir(exist_ok=True)

START_YEAR = 2000
END_YEAR = 2025
MAX_WORKERS = 4

# ---------------- FETCH FUNCTION ----------------
def fetch_airport_daily(airport, coords):
    lat, lon = coords
    point = Point(lat, lon)
    all_data = []
    
    for year in range(START_YEAR, END_YEAR + 1):
        start = datetime(year, 1, 1)
        end = datetime(year, 12, 31)
        try:
            data = Daily(point, start, end).fetch()
            if not data.empty:
                data = data.reset_index().rename(columns = {"time": "date"})
                data['airport'] = airport
                all_data.append(data)
                print(f"✅ Fetched {airport} {year} ({len(data)} rows)")
        except Exception as e:
            print(f"⚠️ Error fetching {airport} {year}: {e}")
        time.sleep(0.05)  # polite delay
    
    if all_data:
        df = pd.concat(all_data)
        out_file = OUTPUT_DIR / f"{airport}.csv"
        df.to_csv(out_file, index=True)
        print(f"✅ Saved {airport} to {out_file}")

# ---------------- MAIN SCRIPT ----------------
def main():
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(fetch_airport_daily, airport, coords)
                   for airport, coords in SWA_AIRPORTS.items()]
        for fut in as_completed(futures):
            fut.result()  # just to catch exceptions

    # Optional: combine all airport CSVs into one master CSV
    all_files = list(OUTPUT_DIR.glob("*.csv"))
    combined = pd.concat((pd.read_csv(f, index_col=0) for f in all_files), ignore_index=True)
    combined.to_csv(OUTPUT_DIR / "SWA_all_airports_daily_2000_2025.csv", index=False)
    print(f"\n✅ All airports combined: {len(combined)} rows")

if __name__ == "__main__":
    main()

✅ Fetched HOU 2000 (366 rows)
✅ Fetched DAL 2000 (366 rows)
✅ Fetched SAT 2000 (366 rows)
✅ Fetched AUS 2000 (366 rows)
✅ Fetched HOU 2001 (365 rows)
✅ Fetched AUS 2001 (365 rows)
✅ Fetched SAT 2001 (365 rows)
✅ Fetched DAL 2001 (365 rows)
✅ Fetched HOU 2002 (365 rows)
✅ Fetched AUS 2002 (365 rows)
✅ Fetched DAL 2002 (365 rows)
✅ Fetched SAT 2002 (365 rows)
✅ Fetched HOU 2003 (365 rows)
✅ Fetched DAL 2003 (365 rows)
✅ Fetched AUS 2003 (365 rows)
✅ Fetched SAT 2003 (365 rows)
✅ Fetched AUS 2004 (366 rows)
✅ Fetched HOU 2004 (366 rows)
✅ Fetched DAL 2004 (366 rows)
✅ Fetched SAT 2004 (366 rows)
✅ Fetched AUS 2005 (365 rows)
✅ Fetched HOU 2005 (365 rows)
✅ Fetched DAL 2005 (365 rows)
✅ Fetched SAT 2005 (365 rows)
✅ Fetched SAT 2006 (365 rows)
✅ Fetched HOU 2006 (365 rows)
✅ Fetched AUS 2006 (365 rows)
✅ Fetched DAL 2006 (365 rows)
✅ Fetched SAT 2007 (365 rows)
✅ Fetched HOU 2007 (365 rows)
✅ Fetched DAL 2007 (365 rows)
✅ Fetched AUS 2007 (365 rows)
✅ Fetched DAL 2008 (366 rows)
✅ Fetched 

In [2]:
df = pd.read_csv("Data/Meteostat_2000-2025.csv")
df.head()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,airport
0,5.0,-1.1,11.1,4.1,0.0,NaN,18.4,NaN,1010.2,383.0,ABQ
1,1.1,-3.9,6.1,0.0,0.0,NaN,10.8,NaN,1011.2,445.0,ABQ
2,-1.1,-5.0,2.2,0.0,0.0,NaN,25.9,NaN,1021.0,588.0,ABQ
3,-2.8,-9.4,3.3,0.0,0.0,NaN,7.2,NaN,1031.3,560.0,ABQ
4,1.1,-7.2,8.9,0.0,0.0,NaN,10.1,NaN,1019.4,565.0,ABQ
